# Read data & clean

In [17]:
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Read data and remove duplicate rows

In [18]:
df = pd.read_csv("data/raw/transaction_data.csv")
df = df.drop_duplicates()
df.head()

,UserId,TransactionId,TransactionTime,ItemCode,ItemDescription,NumberOfItemsPurchased,CostPerItem,Country
0,278166,6355745,Sat Feb 02 12:50:00 IST 2019,465549,FAMILY ALBUM WHITE PICTURE FRAME,6,11.73,United Kingdom
1,337701,6283376,Wed Dec 26 09:06:00 IST 2018,482370,LONDON BUS COFFEE MUG,3,3.52,United Kingdom
2,267099,6385599,Fri Feb 15 09:45:00 IST 2019,490728,SET 12 COLOUR PENCILS DOLLY GIRL,72,0.90,France
3,380478,6044973,Fri Jun 22 07:14:00 IST 2018,459186,UNION JACK FLAG LUGGAGE TAG,3,1.73,United Kingdom
4,-1,6143225,Mon Sep 10 11:58:00 IST 2018,1733592,WASHROOM METAL SIGN,3,3.40,United Kingdom


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536572 entries, 0 to 1081501
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   UserId                  536572 non-null  int64  
 1   TransactionId           536572 non-null  int64  
 2   TransactionTime         536572 non-null  object 
 3   ItemCode                536572 non-null  int64  
 4   ItemDescription         535118 non-null  object 
 5   NumberOfItemsPurchased  536572 non-null  int64  
 6   CostPerItem             536572 non-null  float64
 7   Country                 536572 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 36.8+ MB


`ItemDescription` is the only column that contains null values, replace them with empty strings

In [20]:
df["ItemDescription"] = df["ItemDescription"].fillna("")

Convert `TransactionTime` column to datetime objects

Remove the rows that have invalid `TransactionTime` (times that exceed the current time)

In [21]:
df["TransactionTime"] = pd.to_datetime(df["TransactionTime"], format="%a %b %d %H:%M:%S IST %Y")
df = df[df["TransactionTime"] < datetime.now()]

Remove the rows that have non-positive `NumberOfItemsPurchased` or non-positive `CostPerItem`

In [22]:
df = df[df["NumberOfItemsPurchased"] > 0]
df = df[df["CostPerItem"] > 0]

# Filter outliers

Consider a set of rows that have the same `itemCode`, a row is considered an outlier if:
* `NumberOfItemsPurchased` is anomaly OR
* `CostPerItem` is anomaly

For these two columns, I compute the z-score for each sample in each column sequentially.

A sample is an outlier if either of the two z-scores exceeds the threshold of 3.

In [23]:
from tqdm.notebook import tqdm


def detect_outliers(group, col, threshold):
    """
    Compute z-score of the `col`
    Return a list of index having z-score > threshold
    """
    z_scores = np.abs(stats.zscore(group[col]))
    outliers = group[(z_scores > threshold)]
    return list(outliers.index)


outlier_index = []  
# Group by `ItemCode` --> detect anomaly within each unique item
for item_code, group in tqdm(df.groupby("ItemCode")):
    outlier_index += detect_outliers(group, "NumberOfItemsPurchased", threshold=3)
    outlier_index += detect_outliers(group, "CostPerItem", threshold=3)

  0%|          | 0/3291 [00:00<?, ?it/s]

Remove the outliers from dataset

In [24]:
outlier_index = list(set(outlier_index))
print("Number of outliers:", len(outlier_index))

df_outlier = df[df.index.isin(outlier_index)]
df = df[~df.index.isin(outlier_index)]

Number of outliers: 15359


# Statistics

Calculate the number of Items purchased and prices in each month

In [25]:
df["MonthYear"] = df["TransactionTime"].apply(lambda dt: f"{dt.year}-{dt.month}")

df.groupby("MonthYear").agg({
    "NumberOfItemsPurchased": "sum",
    "CostPerItem": "sum"
})

,NumberOfItemsPurchased,CostPerItem
MonthYear,,
2018-10,954111,157763.50
2018-11,947163,179260.37
2018-12,1470798,274831.83
2018-2,663951,185927.34
2018-3,465807,127527.54
2018-4,634248,159069.61
2018-5,710157,167814.89
2018-6,820260,184092.25
2018-7,833658,176661.96


Calculate the number of items purchased for each userID in 30 days for each day 

In [26]:
df.sort_values(by="TransactionTime", inplace=True)

In [27]:
df["NumberOfItemsPurchased_30days"] = \
    df.set_index("TransactionTime").groupby("UserId")["NumberOfItemsPurchased"]\
        .rolling("30D").sum().reset_index().astype(int)["NumberOfItemsPurchased"]

In [28]:
df.head()

,UserId,TransactionId,TransactionTime,ItemCode,ItemDescription,NumberOfItemsPurchased,CostPerItem,Country,MonthYear,NumberOfItemsPurchased_30days
577478,374850,5900015,2018-02-12 04:26:00,1787583,WHITE HANGING HEART T-LIGHT HOLDER,18,3.52,United Kingdom,2018-2,NaN
528742,374850,5900015,2018-02-12 04:26:00,1764609,KNITTED UNION FLAG HOT WATER BOTTLE,18,4.68,United Kingdom,2018-2,NaN
55169,374850,5900015,2018-02-12 04:26:00,477792,SET 7 BABUSHKA NESTING BOXES,6,10.56,United Kingdom,2018-2,66072.0
323263,374850,5900015,2018-02-12 04:26:00,1492113,WHITE METAL LANTERN,18,4.68,United Kingdom,2018-2,4839.0
412255,374850,5900015,2018-02-12 04:26:00,1764609,RED WOOLLY HOTTIE WHITE HEART.,18,4.68,United Kingdom,2018-2,318.0


In [29]:
df.to_csv("data/cleaned/transaction_data.csv", index=False)